In [1]:
import pandas as pd
from pycaret.classification import *
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_20newsgroups

In [2]:
from pycaret.datasets import get_data
dataset = get_data('juice')

print(dataset.shape)

,Id,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,1,CH,237,1,1.75,1.99,0.00,0.0,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1
1,2,CH,239,1,1.75,1.99,0.00,0.3,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1
2,3,CH,245,1,1.86,2.09,0.17,0.0,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1
3,4,MM,227,1,1.69,1.69,0.00,0.0,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1
4,5,CH,228,7,1.69,1.69,0.00,0.0,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0


(1070, 19)


In [3]:
py = setup(
    data=dataset,
    target='STORE',
    session_id=123,
    experiment_name=None,
    # text_features=["Purchase"],
    ignore_features=["Id"],
    keep_features=["SalePriceCH"],
    # preprocess=False,
    # imputation_type='simple',
    # numeric_iterative_imputer='lightgbm',
    # categorical_iterative_imputer='lightgbm',
    # max_encoding_ohe=5,
    # normalize=True,
    # remove_outliers=True,
    # outliers_method="lof",
    # outliers_threshold=0.5,
    # remove_multicollinearity=True,
    # multicollinearity_threshold=0.8,
    # bin_numeric_features=["PriceMM"],
    # feature_selection=True,
    # feature_selection_method="boruta",
    # feature_selection_estimator="rf",
    # n_features_to_select=7,
    # custom_pipeline=[("pca", PCA(n_components=8))],
    system_log=False,
)

In [4]:
a = py._internal_pipeline
print(a)
a.fit(py.X, py.y)
X, y = a.transform(py.X, py.y)
print(X)
print(X.shape)

Pipeline(steps=[('numerical_imputer',
                 TransfomerWrapper(columns=['WeekofPurchase', 'StoreID', 'PriceCH', 'PriceMM', 'DiscCH', 'DiscMM', 'SpecialCH', 'SpecialMM', 'LoyalCH', 'SalePriceMM', 'SalePriceCH', 'PriceDiff', 'PctDiscMM', 'PctDiscCH', 'ListPriceDiff'], transformer=SimpleImputer(fill_value=0))),
                ('categorical_imputer',
                 TransfomerWrapper(columns=['Purchase', 'Store7'], transformer=SimpleImputer(fill_value='not_available', strategy='constant'))),
                ('ordinal_encoding',
                 TransfomerWrapper(columns=['Purchase', 'Store7'], transformer=OrdinalEncoder(handle_missing='return_nan', mapping=[{'col': 'Purchase', 'mapping': {'CH': 1, 'MM': 0}}, {'col': 'Store7', 'mapping': {'No': 1, 'Yes': 0}}])))])
      Purchase  WeekofPurchase  StoreID  PriceCH  PriceMM  DiscCH  DiscMM  \
0          0.0           255.0      7.0     1.86     2.18    0.00     0.0   
1          0.0           261.0      2.0     1.86     2.18    0.0

In [7]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.5235,0.8218,0.4479,0.4636,0.4691,0.3973,0.4372,0.6040
xgboost,Extreme Gradient Boosting,0.5221,0.8109,0.4501,0.4697,0.4632,0.3970,0.4395,0.1250
lightgbm,Light Gradient Boosting Machine,0.5221,0.8163,0.4501,0.4698,0.4634,0.3968,0.4390,0.1030
lr,Logistic Regression,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0080
knn,K Neighbors Classifier,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0090
nb,Naive Bayes,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0070
dt,Decision Tree Classifier,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0090
svm,SVM - Linear Kernel,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0090
ridge,Ridge Classifier,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0090
rf,Random Forest Classifier,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0070


In [8]:
best_model